# Домашнее задание 1. 

-  "Оптимальность" наивного Байесовского классификатора
- Оценка "важности" признаков (feature importance)

Для тех кто не знаком с библиотекой pandas я рекомендую вот эти два практических туториала:

-  https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb (1-3 часа на прохождение, 30 вопросов с видео разборами)
- https://github.com/guipsamora/pandas_exercises/ 11 топиков покрывающих весь основной функционал библиотеки


## 1. Наивный Байес

Ваша задача сравнить вероятности получаемые наивным байесовским классификатором с вероятностями получаемыми если не делать предположения о независимости признаков. Мы будем делать это для 2 признаков рассмотренных на семинаре: Пол и Класс билета (пассажира).

In [29]:
import pandas as pd

In [35]:
df = pd.read_csv('../data/titanic_data.csv')

In [3]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

## 1.1 Разделим данные на тренировочную и тестовую части

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

## 1.2 Реализуйте функцию `get_probability_survived_bayes`

Которая считает честные вероятности выживания при условии заданного класса билета и пола:

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{Passenger Class}, \text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}|\text{Passenger Class}) p(\text{Passenger Class})}
$$

## Помните что оценка вероятностей всегда должна происходить на тренировчной выборке.

In [13]:
from typing import Tuple

In [14]:
# Hint 1. Используйте pandas.groupby
# Hint 2. Вы возвращаете tuple p_not_survived, p_survived, в котором
# вероятность выжить находится на второй позиции, чтобы когда вы берете аргмакс получать метки классов 0/1 (не выжил/выжил)

def get_probability_survived_bayes(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    ...
    


## 1.3 Реализуйте функцию `get_probability_survived_bayes_naive`

Которая оценивает вероятность выжить в предположении о независимости признаков

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}) p(\text{Passenger Class})}
$$

In [21]:
def get_probability_survived_bayes_naive(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns tuple(p_not_survived, p_survived), 
    under naive assumption of features independence'''
    ...
    
# Hint 1. Поскольку вы делаете предположении о независимости признаков, то p(y=1|...) + p(y=0| ...) некорректно отнормированы
# и не будут суммироваться в 1. Не забудьте выполнить необходимую нормировку.

## 1.4 Используйте ваши функции для того чтобы сделать предсказание на тесте


1. Для всех 6 уникальных комбинаций значений признаков сравните вероятности принадлежности классу 1
 - насколько они отличаются?
 - влияют ли эти отличия на предсказание класса объекта, при пороге $p = 0.5$?
2. Посчитайте точность (accuracy) на тренировочной и тестовой выборке для двух подходов, какой подход дает более высокую точность? (используйте порог $p = 0.5$).
3. С вашей точки зрения в данной задаче следует максимизировать Recall или Precision? Объясните свой ответ.
4. В зависимости от вашего ответа на предыдущий вопрос, какой классификатор и какой порог вероятности необходимо выбрать. Сравните матрицу ошибок (confusion matrix) для найденного порога и порога $p = 0.5$ и прокомментируйте отличия.


In [ ]:
y_pred_proba = []

for _, (sex, pclass, _) in tqdm(X_test.iterrows()):
    y_pred_proba.append(get_probability_survived_bayes_naive(pclass, sex))

## 1.5 Обучите имплементацию `MultinomialNB` из `sklearn`

1. Тот же вопрос что и 1.4.1, прокомментируйте отличия в оценке вероятности ваших имлементаций и имплементации
`MultinomialNB`. Влияют ли они на предсказание классов при стандартном пороге вероятности? Заполните таблицу `probabilities`.

In [25]:
probabilities = pd.DataFrame()
probabilities['Sex'] = [0,0,0,1,1,1]
probabilities['Passenger Class'] = [1,2,3,1,2,3]
probabilities['Bayes'] = [0]*6
probabilities['BayesNaive'] = [0]*6
probabilities['MultinomialNB'] = [0]*6

probabilities

,Sex,Passenger Class,Bayes,BayesNaive,MultinomialNB
0,0,1,0,0,0
1,0,2,0,0,0
2,0,3,0,0,0
3,1,1,0,0,0
4,1,2,0,0,0
5,1,3,0,0,0


## 2.1 Используя MultinomialNB обучите классификатор на бОльшем количестве признаков

1. Подготовьте признаки 
- 'Passenger Class',
- 'Sex',
- 'Age',
- 'No of Siblings or Spouses on Board',
- 'No of Parents or Children on Board',
- 'Port of Embarkation'

для обучения:
- закодируйте столбец `Port of Embarkation` по аналогии с `Passenger Class`
- заполните пропуски в столбце `Age` средним возрастом в датасете используйте `pd.DataFrame.fillna(...)`
- заполните пропуске в столбце `Port of Embarkation` самым частым портом посадки
- для разбиения на тренировчную и тестовую часть данных используйте `train_size=0.66` и `random_state=5`

2. обучите MultinomialNB модель на тренировочной части данных
2. Как изменилась точность вашей модели на тестовой части данных по сравнению с моделью обученной на 2 признаках?
3. Что означает признак `Life Boat`? Следуют ли на ваш взгляд включать его в список признаков для обучения модели? 
- Опишите сценарий (использования модели) в котором его следует включать в список признаков. 
- Опишите сценарий (использования модели) в котором его не следует включать в список признаков.
5. Какая ошибка содержится в предложенной последовательности действий в п. 2.1? 

In [62]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})


X['Age'] = df['Age']

X['No of Siblings or Spouses on Board'] = df['No of Siblings or Spouses on Board']
X['No of Parents or Children on Board'] = df['No of Parents or Children on Board']

X['Port of Embarkation'] = df['Port of Embarkation'].map(
{
    'Southampton':1,
    'Cherbourg':2,
    'Queenstown':3
    
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

In [63]:
columns = ['Passenger Class', 'Sex', 'Age', 'No of Siblings or Spouses on Board',
 'No of Parents or Children on Board', 'Port of Embarkation']

In [64]:
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Port of Embarkation'] = X['Port of Embarkation'].fillna(X['Port of Embarkation'].median())

In [65]:
X['Age'] = X['Age'] // 10

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

In [124]:
clf = MultinomialNB()

clf.fit(X_train.drop('Port of Embarkation', axis=1), y_train)
# clf.fit(X_train, y_train)

MultinomialNB()

In [125]:
y_train_pred = clf.predict(X_train.drop('Port of Embarkation', axis=1))
y_test_pred = clf.predict(X_test.drop('Port of Embarkation', axis=1))

# y_train_pred = clf.predict(X_train)
# y_test_pred = clf.predict(X_test)

In [126]:
print(f'Model performance on train data: {accuracy_score(y_train, y_train_pred)}')

Model performance on train data: 0.7809965237543454


In [127]:
print(f'Model performance on test data: {accuracy_score(y_test, y_test_pred)}')

Model performance on test data: 0.7623318385650224


In [128]:
X_train

,Sex,PassengerClass,Age,No of Siblings or Spouses on Board,No of Parents or Children on Board,Port of Embarkation
615,0,3,2.0,0,0,1.0
287,0,1,6.0,0,0,1.0
720,0,3,2.0,0,0,3.0
222,0,1,2.0,0,0,2.0
244,0,1,3.0,0,0,2.0
...,...,...,...,...,...,...
73,1,1,2.0,0,0,1.0
1142,0,3,0.0,4,1,3.0
998,0,3,2.0,0,0,2.0
206,0,1,4.0,2,0,3.0


In [81]:
y.mean()

0.3819709702062643

In [46]:
print(f'Model performance on train data: {accuracy_score(y_train, y_train_pred)}')

Model performance on train data: 0.7844727694090382


In [47]:
print(f'Model performance on test data: {accuracy_score(y_test, y_test_pred)}')

Model performance on test data: 0.7668161434977578


## 2.2. Оценка важности признаков

Мы воспользуемся простым (и не очень хорошим^[1]) методом оценки важности признаков:

1. Воспользуйтесь моделью из п.2. Качество модели на тестовой части данных это ваш бейзлайн
2. Исключая по 1 признаку из `X` обучайте модель `MultinomialNB` и измеряйте ее точность на тестовой части данных
3. Сохраняйте величину падения или роста качества по сравнению с бейзлайном. Если точность упала значит признак "полезный", если выросло - "вредный". Абсолютная величина падения или роста в данном случае не так важна, важен только порядок признаков. Такая оценка называется качественной (в противовес количественной).
4. Прокомментируйте полученный результат, насколько он согласуется с вашей интуицией?
5. В чем на ваш взляд могут быть преимущества использования мЕньшего числа признаков при обучении моделей? (не обязательно в контексте данной задачи)


[1] а впрочем какие из них хороши?

## 3. Подготовьте репорт о выполненном задании

Подготовьте текстовый репорт о выполенном домашнем задании. Ваш репорт должен включать ответы на вопросы:
- 1.4.1-1.4.4
- 1.5.1
- 2.1.3-2.1.5
- 2.2.3-2.2.5

Вы можете вставлять в текст таблицы и/или графики на ваше усмотрение (для п. 1.5.1 обязательно вставьте таблицу или скриншот таблицы), текст репорта должен быть в двухколоночном формате не более 2 страниц 12 кеглем Times New Roman (**крайне** желательно 1 страница), см шаблон https://docs.google.com/document/d/1YqnO0OFbTR7XgzmdWryQF22J1kl9Fg7hSlNbqa6Jbiw/edit?usp=sharing

Подготовленный репорт и jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма `[iitp-intro-ds-2023-ha1-Surname]`, например `[iitp-intro-ds-2023-Kurmukov]` до 23:59:59 МСК 25.02.2023 .

In [6]:
from functools import lru_cache

@lru_cache
def get_probability_survived_bayes(pclass, sex):   
    sub = X_train.groupby(['Sex', 'PassengerClass'])['y'].mean().reset_index()
    p = sub.query('Sex==@sex').query('PassengerClass==@pclass')['y'].values[0]
    return 1-p, p

@lru_cache
def get_probability_survived_bayes_naive(pclass, sex):
    # P(Sex=1|y=1) * P(Passenger Class=1| y=1) * P(y=1) / P(Sex=1) * P(Passenger Class=1)
    
    # P(Sex=1|y=1)
    p1 = X_train.query('y==1')['Sex'].mean()
    p10 = X_train.query('y==0')['Sex'].mean()
    if sex == 0:
        p1 = 1 - p1
        p10 = 1 - p10
        
    # P(Passenger Class=1| y=1)
    p2 = X_train.query('y==1')['PassengerClass'].value_counts()[pclass] / X_train.query('y==1')['PassengerClass'].count()
    p20 = X_train.query('y==0')['PassengerClass'].value_counts()[pclass] / X_train.query('y==0')['PassengerClass'].count()
    
    # P(y=1)
    p3 = X_train['y'].mean()
    p30 = 1 - p3
    
    # P(Sex=1)
    p4 = X_train['Sex'].mean()
    if sex == 0:
        p4 = 1 - p4
    
    # P(Passenger Class=1)
    p5 = X_train['PassengerClass'].value_counts()[pclass] / X_train['PassengerClass'].count()
    
#     print(p1, p2, p3, p4, p5)
    
    p_1 = p1 * p2 * p3 / (p4 * p5)
    p_0 = p10 * p20 * p30 / (p4 * p5)
    return p_0 / (p_1 + p_0), p_1 / (p_1 + p_0)

In [7]:
clf = MultinomialNB(alpha=0)

clf.fit(X.drop('y', axis=1), y)

/home/anvar/anaconda3/lib/python3.8/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


MultinomialNB(alpha=0)

In [9]:
# accuracy_score(y, clf.predict(X.drop('y', axis=1)))

In [10]:
from tqdm import tqdm
import numpy as np

In [196]:
y_pred = []
for _, (sex, pclass, _) in tqdm(X_test.iterrows()):
    y_pred.append(np.argmax(get_probability_survived_bayes_naive(pclass, sex)))

446it [00:00, 14598.45it/s]


In [197]:
accuracy_score(y_test, y_pred)

0.7645739910313901

In [198]:
y_pred = []
for _, (sex, pclass, _) in tqdm(X_test.iterrows()):
    y_pred.append(np.argmax(get_probability_survived_bayes(pclass, sex)))

446it [00:00, 13522.09it/s]


In [200]:
accuracy_score(y_test, y_pred)

0.7645739910313901

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score

In [187]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

In [16]:
clf = MultinomialNB()

clf.fit(X_train.drop('y', axis=1), y_train)

MultinomialNB()

In [17]:
y_train_pred = clf.predict(X_train.drop('y', axis=1))
y_test_pred = clf.predict(X_test.drop('y', axis=1))

In [20]:
print(f'Model performance on train data: {accuracy_score(y_train, y_train_pred)}')

Model performance on train data: 0.7879490150637312


In [21]:
print(f'Model performance on test data: {accuracy_score(y_test, y_test_pred)}')

Model performance on test data: 0.7645739910313901
